# Airbyte Question Answering | Airbyte 質問応答

このノートブックは、構造化されたデータに対して質問応答を行う方法を示しており、この場合は `AirbyteStripeLoader` を使用しています。

> This notebook shows how to do question answering over structured data, in this case using the `AirbyteStripeLoader`.

Vectorstoresは、計算、グルーピング、および構造化されたデータのフィルタリングを必要とする質問に答えるのが難しいことがよくあります。そこで、このタイプの質問に対応するために`pandas`データフレームを使用するというのが、高レベルなアイデアです。

> Vectorstores often have a hard time answering questions that requires computing, grouping and filtering structured data so the high level idea is to use a `pandas` dataframe to help with these types of questions.




1. Airbyteを使用してStripeからデータを読み込むには、`record_handler`パラメータを使用してデータローダーからJSONを返すように設定します。

   > Load data from Stripe using Airbyte. user the `record_handler` paramater to return a JSON from the data loader.





In [ ]:
import os

import pandas as pd
from langchain.agents import AgentType, create_pandas_dataframe_agent
from langchain.chat_models.openai import ChatOpenAI
from langchain.document_loaders.airbyte import AirbyteStripeLoader

stream_name = "customers"
config = {
    "client_secret": os.getenv("STRIPE_CLIENT_SECRET"),
    "account_id": os.getenv("STRIPE_ACCOUNT_D"),
    "start_date": "2023-01-20T00:00:00Z",
}


def handle_record(record: dict, _id: str):
    return record.data


loader = AirbyteStripeLoader(
    config=config,
    record_handler=handle_record,
    stream_name=stream_name,
)
data = loader.load()

2. データを`pandas`データフレームに渡してください。

   > Pass the data to `pandas` dataframe.





In [ ]:
df = pd.DataFrame(data)

3. `df`データフレームを`create_pandas_dataframe_agent`に渡して実行してください。

   > Pass the dataframe `df` to the `create_pandas_dataframe_agent` and invoke





In [ ]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-4"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

4. エージェントを実行する

   > Run the agent





In [ ]:
output = agent.run("How many rows are there?")